## Importing libraries

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Loading files and data manipulation

In [15]:
females = pd.read_json('females.json')
males = pd.read_json('males.json')

profiles = females.append(males)

profiles

,id,gender,bio,interests,anthem,prompts
0,1,1,Instagram- leonor_ortizzz,"[Coffee, Activism, Foodie, Music, Movies]",,"[, , ]"
1,2,1,"lover of avocados, coconuts, and one too many ...","[Fashion, Karaoke, Museum, Brunch, Volunteering]","OPEN A WINDOW (feat. Tyler, The Creator)","[Aries, , ]"
2,3,1,mommy issue haver costco hotdog enthusiast goo...,"[Foodie, Music, Karaoke, Dog lover, Comedy]",Fair Trade (with Travis Scott),"[Libra, Dog, Social smoker]"
3,4,1,Follow my new insta page: angelisse_req Im 18 ...,"[Foodie, Tattoos, Sports, Astrology, Spiritual...",,"[, , ]"
4,5,1,"they/she. from bushwick, and 420 friendly. :) ...","[Photography, Black Lives Matter, Movies, Musi...",Wolfcat,"[Scorpio, Cat, ]"
5,6,1,Dont know what to put here .. Ask me anything,"[Picnicking, Fashion, Road Trips, Music, Spiri...",Kiss Me More (feat. SZA),"[, , ]"
6,7,1,howdy i delete this app a lot so if i disappea...,"[Writer, Reading, Gamer, Fashion, Spirituality]",Industry Plant,"[Pisces, Reptile, Non-smoker]"
7,8,1,,"[Sports, Dog lover, Athlete, Working out, Movies]",,"[, , ]"
8,9,1,:) ig: katy.wsl bin hier nicht so aktiv. krank...,"[Gin Tonic, Sushi, Movies, Working out, Cooking]",The Cure,"[Gemini, , Non-smoker]"
9,10,1,dm me on instagram: torineomi,"[Travel, Tattoos, Fashion, Art, Second-hand ap...",L$D,"[, , ]"
